# Testing notebook

In [74]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [75]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [76]:
from computions import *

### get_describe_nominal function test

Basic situation

In [77]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,object_variable,1.0,1.0%,2.0,2.0%,0.0,0.0%
4.0,111,79,71.171171,1.0,0.900901,31.0,27.927928
1.0,107,20,18.691589,0.0,0.000000,87.0,81.308411
7.0,106,78,73.584906,27.0,25.471698,1.0,0.943396
9.0,103,42,40.776699,61.0,59.223301,0.0,0.000000
6.0,98,90,91.836735,5.0,5.102041,3.0,3.061224
8.0,97,60,61.855670,37.0,38.144330,0.0,0.000000
3.0,95,56,58.947368,0.0,0.000000,39.0,41.052632
2.0,91,33,36.263736,0.0,0.000000,58.0,63.736264
5.0,82,70,85.365854,4.0,4.878049,8.0,9.756098
10.0,63,12,19.047619,51.0,80.952381,0.0,0.000000


Data frame with emptys

In [78]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['object_variable'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [79]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.5219025443529544, 2.0: 0.7708845208845209, 0.0: 0.26542299612152553}
{'category 1': 0.5219025443529544, 'category 2': 0.7708845208845209, 'cetegory 0': 0.26542299612152553}
{'category 1': 0.5263438155136269, 'category 2': 0.7654457574730844, 'cetegory 0': 0.2664345207284363}
{1.0: 0.7345770038784745, 0.0: 0.26542299612152553}


AUC nominal

In [80]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

{'category 1': 0.7968868334971559, 'category 2': 0.9175186917122401, 'cetegory 0': 0.9272079023917259}
AUC bynary
{1.0: 0.927207902391726, 0.0: 0.9272079023917259}
AUC with emptys
{'category 1': 0.5854077684959699, 'category 2': 0.916822563906489, 'cetegory 0': 0.07227729013347624}


Get publish AUC for any predictor type

In [81]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)


with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{1.0: {'AUC': 0.7968868334971559, 'rel_type': 1, 'GINI': 0.5937736669943119}, 2.0: {'AUC': 0.9175186917122401, 'rel_type': 1, 'GINI': 0.8350373834244802}, 0.0: {'AUC': 0.9272079023917259, 'rel_type': 1, 'GINI': 0.8544158047834518}}
moninal predictor with table
{'category 1': {'AUC': 0.7968868334971559, 'rel_type': 1, 'GINI': 0.5937736669943119}, 'category 2': {'AUC': 0.9175186917122401, 'rel_type': 1, 'GINI': 0.8350373834244802}, 'cetegory 0': {'AUC': 0.9272079023917259, 'rel_type': 1, 'GINI': 0.8544158047834518}}
numeric predictor
{1.0: {'AUC': 0.5219025443529544, 'rel_type': 1, 'GINI': 0.04380508870590871}, 2.0: {'AUC': 0.7708845208845209, 'rel_type': 1, 'GINI': 0.5417690417690417}, 0.0: {'AUC': 0.7345770038784745, 'rel_type': -1, 'GINI': 0.46915400775694893}}


## Writing to excel

Adding a new sheet

In [95]:
xl_writer.close()

In [100]:
xl_writer = pd.ExcelWriter("test_result/test.xlsx",engine='xlsxwriter')


for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

test


In [10]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.write_to_book(xl_writer)

In [13]:
predictos_data = my_cpp.get_predictors_data()
predictos_data['object_variable']

{'name': 'object_variable',
 'emptys_count': 0,
 'predictor_type': 'nominal',
 'describe_table':       object_variable  1.0       1.0%   0.0       0.0%   2.0       2.0%
 6.0               123  112  91.056911   7.0   5.691057   4.0   3.252033
 1.0               113   22  19.469027  91.0  80.530973   0.0   0.000000
 8.0               110   80  72.727273   0.0   0.000000  30.0  27.272727
 2.0               107   21  19.626168  86.0  80.373832   0.0   0.000000
 9.0               102   51  50.000000   0.0   0.000000  51.0  50.000000
 3.0                94   48  51.063830  46.0  48.936170   0.0   0.000000
 7.0                91   80  87.912088   1.0   1.098901  10.0  10.989011
 5.0                87   79  90.804598   7.0   8.045977   1.0   1.149425
 4.0                82   63  76.829268  19.0  23.170732   0.0   0.000000
 0.0                49    4   8.163265  45.0  91.836735   0.0   0.000000
 10.0               42    7  16.666667   0.0   0.000000  35.0  83.333333,
 'AUC_data': {1.0: {'AUC': 

## Saving the results

In [7]:
xl_writer.close()